# Text Summarization with DialogSum

**Module 02 | Notebook 3 of 3**

In this notebook, we'll fine-tune a sequence-to-sequence model for dialogue summarization.

## Learning Objectives

By the end of this notebook, you will be able to:
1. Work with encoder-decoder models (T5, BART)
2. Prepare data for seq2seq tasks
3. Use ROUGE metrics for evaluation
4. Generate summaries from dialogue

---

In [ ]:
%%capture
!pip install transformers datasets accelerate evaluate rouge-score nltk

In [ ]:
import torch
from transformers import (
    AutoTokenizer, AutoModelForSeq2SeqLM,
    Seq2SeqTrainingArguments, Seq2SeqTrainer,
    DataCollatorForSeq2Seq
)
from datasets import load_dataset
import evaluate
import numpy as np
import nltk
import warnings
warnings.filterwarnings('ignore')

# Download NLTK data for sentence tokenization
nltk.download('punkt', quiet=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

---

## Understanding Sequence-to-Sequence Models

### How Summarization Works

```
Input (Dialogue):
┌─────────────────────────────────────────────┐
│ Person A: Hi, how are you?                  │
│ Person B: I'm great! Just got promoted.     │
│ Person A: Congratulations! That's amazing!  │
│ Person B: Thanks! Let's celebrate tonight.  │
└─────────────────────────────────────────────┘
                    │
                    ▼
            ┌───────────────┐
            │    ENCODER    │  (Understands input)
            └───────────────┘
                    │
                    ▼
            ┌───────────────┐
            │    DECODER    │  (Generates output)
            └───────────────┘
                    │
                    ▼
Output (Summary):
┌─────────────────────────────────────────────┐
│ Person B got promoted and they plan to      │
│ celebrate tonight.                          │
└─────────────────────────────────────────────┘
```

### Popular Summarization Models

| Model | Parameters | Best For |
|-------|------------|----------|
| T5-small | 60M | Quick experiments |
| T5-base | 220M | Good balance |
| BART-base | 140M | News articles |
| FLAN-T5 | Various | Instruction-tuned |

---

## Load the DialogSum Dataset

In [ ]:
# Load DialogSum dataset
dataset = load_dataset("knkarthick/dialogsum")

print("Dataset structure:")
print(dataset)
print(f"\nTrain examples: {len(dataset['train'])}")
print(f"Validation examples: {len(dataset['validation'])}")
print(f"Test examples: {len(dataset['test'])}")

In [ ]:
# Explore the data
print("Sample dialogue and summary:")
print("=" * 60)
example = dataset['train'][0]

print("DIALOGUE:")
print(example['dialogue'])
print("\nSUMMARY:")
print(example['summary'])
print("\nTOPIC:")
print(example.get('topic', 'N/A'))

In [ ]:
# Analyze lengths
dialogue_lengths = [len(d.split()) for d in dataset['train']['dialogue']]
summary_lengths = [len(s.split()) for s in dataset['train']['summary']]

print("Length Statistics (words):")
print(f"\nDialogues:")
print(f"  Mean: {np.mean(dialogue_lengths):.0f}")
print(f"  Max: {max(dialogue_lengths)}")

print(f"\nSummaries:")
print(f"  Mean: {np.mean(summary_lengths):.0f}")
print(f"  Max: {max(summary_lengths)}")

# Compression ratio
ratios = [s/d for s, d in zip(summary_lengths, dialogue_lengths) if d > 0]
print(f"\nCompression ratio: {np.mean(ratios):.1%}")

---

## Data Preparation for Seq2Seq

In [ ]:
# Load model and tokenizer
model_name = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# T5 uses a prefix for summarization
prefix = "summarize: "

# Tokenization parameters
max_input_length = 512
max_target_length = 128

In [ ]:
def preprocess_function(examples):
    """Tokenize inputs and targets for seq2seq training."""
    # Add prefix to inputs
    inputs = [prefix + doc for doc in examples['dialogue']]
    
    # Tokenize inputs
    model_inputs = tokenizer(
        inputs,
        max_length=max_input_length,
        truncation=True,
        padding=True
    )
    
    # Tokenize targets (summaries)
    labels = tokenizer(
        examples['summary'],
        max_length=max_target_length,
        truncation=True,
        padding=True
    )
    
    model_inputs['labels'] = labels['input_ids']
    
    return model_inputs

In [ ]:
# Use smaller subsets for faster training
train_size = 1000
val_size = 200

train_data = dataset['train'].shuffle(seed=42).select(range(train_size))
val_data = dataset['validation'].shuffle(seed=42).select(range(val_size))

# Tokenize
train_tokenized = train_data.map(preprocess_function, batched=True)
val_tokenized = val_data.map(preprocess_function, batched=True)

print(f"Training samples: {len(train_tokenized)}")
print(f"Validation samples: {len(val_tokenized)}")

---

## Understanding ROUGE Metrics

ROUGE (Recall-Oriented Understudy for Gisting Evaluation) measures summary quality:

| Metric | What it Measures |
|--------|------------------|
| ROUGE-1 | Unigram (word) overlap |
| ROUGE-2 | Bigram (2-word) overlap |
| ROUGE-L | Longest common subsequence |

Higher scores = better summaries

In [ ]:
# Load ROUGE metric
rouge = evaluate.load("rouge")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    
    # Decode predictions
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    
    # Replace -100 in labels (padding)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Add newlines for ROUGE calculation
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    # Compute ROUGE scores
    result = rouge.compute(
        predictions=decoded_preds,
        references=decoded_labels,
        use_stemmer=True
    )
    
    return {
        'rouge1': result['rouge1'],
        'rouge2': result['rouge2'],
        'rougeL': result['rougeL']
    }

---

## Model Training

In [ ]:
# Load model
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

total_params = sum(p.numel() for p in model.parameters())
print(f"Model parameters: {total_params:,}")

In [ ]:
# Seq2Seq training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./summarization_model",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=3e-5,
    weight_decay=0.01,
    warmup_ratio=0.1,
    
    # Generation during evaluation
    predict_with_generate=True,
    generation_max_length=max_target_length,
    
    # Evaluation
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="rougeL",
    
    # Logging
    logging_steps=50,
    
    # Performance
    fp16=torch.cuda.is_available(),
    report_to="none"
)

# Data collator for seq2seq
data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model
)

# Create trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized,
    eval_dataset=val_tokenized,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

In [ ]:
# Train
print("Starting training...")
print("=" * 50)
trainer.train()
print("\nTraining complete!")

---

## Evaluation

In [ ]:
# Evaluate
eval_results = trainer.evaluate()

print("Evaluation Results:")
print("=" * 40)
print(f"ROUGE-1: {eval_results['eval_rouge1']:.2%}")
print(f"ROUGE-2: {eval_results['eval_rouge2']:.2%}")
print(f"ROUGE-L: {eval_results['eval_rougeL']:.2%}")

---

## Generate Summaries

In [ ]:
from transformers import pipeline

# Create summarization pipeline
summarizer = pipeline(
    "summarization",
    model=model,
    tokenizer=tokenizer,
    device=0 if torch.cuda.is_available() else -1
)

In [ ]:
# Test on examples from the test set
test_examples = dataset['test'].select(range(3))

print("Generated Summaries:")
print("=" * 60)

for i, example in enumerate(test_examples):
    dialogue = example['dialogue']
    reference = example['summary']
    
    # Generate summary
    generated = summarizer(
        prefix + dialogue,
        max_length=128,
        min_length=20,
        do_sample=False
    )[0]['summary_text']
    
    print(f"\n--- Example {i+1} ---")
    print(f"\nDIALOGUE (truncated):")
    print(dialogue[:300] + "...")
    print(f"\nREFERENCE SUMMARY:")
    print(reference)
    print(f"\nGENERATED SUMMARY:")
    print(generated)
    print("-" * 60)

In [ ]:
# Test on custom dialogue
custom_dialogue = """
#Person1#: Hey Sarah, have you finished the project report?
#Person2#: Almost done! Just need to add the conclusion section.
#Person1#: Great! The client meeting is tomorrow at 2 PM.
#Person2#: I'll have it ready by noon so you can review it.
#Person1#: Perfect. Also, can you include the budget projections?
#Person2#: Already added those. I also updated the timeline chart.
#Person1#: You're a lifesaver! Thanks so much.
#Person2#: No problem! See you tomorrow.
"""

summary = summarizer(
    prefix + custom_dialogue,
    max_length=100,
    min_length=20,
    do_sample=False
)[0]['summary_text']

print("Custom Dialogue Summarization:")
print("=" * 50)
print("\nINPUT:")
print(custom_dialogue)
print("\nSUMMARY:")
print(summary)

---

## Generation Parameters

You can control how summaries are generated:

In [ ]:
test_dialogue = dataset['test'][0]['dialogue']

# Different generation strategies
print("Generation Strategies Comparison:")
print("=" * 50)

# Greedy (default)
greedy = summarizer(prefix + test_dialogue, max_length=100, do_sample=False)[0]['summary_text']
print(f"\nGreedy: {greedy}")

# Beam search
beam = summarizer(prefix + test_dialogue, max_length=100, num_beams=4, do_sample=False)[0]['summary_text']
print(f"\nBeam Search (4): {beam}")

# Sampling
sampled = summarizer(prefix + test_dialogue, max_length=100, do_sample=True, top_k=50, temperature=0.7)[0]['summary_text']
print(f"\nSampling (temp=0.7): {sampled}")

### Generation Parameters Explained

| Parameter | Effect |
|-----------|--------|
| `max_length` | Maximum output tokens |
| `min_length` | Minimum output tokens |
| `num_beams` | Beam search width (higher = better but slower) |
| `do_sample` | Enable random sampling |
| `temperature` | Randomness (0=deterministic, 1=default, >1=more random) |
| `top_k` | Limit sampling to top K tokens |
| `top_p` | Nucleus sampling threshold |

---

## 🎯 Student Challenge

### Challenge: Fine-tune for News Summarization

In [ ]:
# TODO: Adapt this notebook for news summarization
# 1. Load the CNN/DailyMail dataset: load_dataset("cnn_dailymail", "3.0.0")
# 2. The columns are 'article' (input) and 'highlights' (summary)
# 3. Update the preprocess_function
# 4. Train and evaluate
# 5. Compare ROUGE scores

# Hint: CNN/DailyMail articles are longer, you may need max_input_length=1024

# Your solution:


---

## Key Takeaways

1. **Seq2Seq models** have separate encoder and decoder components
2. **T5 uses prefixes** ("summarize:", "translate:") to specify tasks
3. **ROUGE metrics** measure n-gram overlap between generated and reference summaries
4. **Generation parameters** control output quality and diversity
5. **Beam search** usually produces better results than greedy decoding

---

## Next Steps

Continue to Module 03: **Model Optimization**
- `03_Model_Optimization/01_intro_to_optimization.ipynb`